In [3]:
import configparser
from pathlib import Path
import pandas as pd
import pandas.io.sql as pandasql
from psycopg2 import connect
import numpy
from psycopg2.extras import execute_values
from datetime import datetime
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import copy
import rick
import geopandas as gpd
import seaborn as sns
import shapely
import matplotlib.patches as patches
import matplotlib.ticker as ticker
from setuptools import setup, find_packages
from math import factorial
from simanneal import Annealer
from itertools import islice 
import matplotlib.patches as patches
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
from itertools import combinations, chain
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [ ]:
a = 0
month = ['congestion.metrics_v5_201901','congestion.metrics_v5_201902', 'congestion.metrics_v5_201903', 'congestion.metrics_v5_201904', 'congestion.metrics_v5_201905', 'congestion.metrics_v5_201906', 'congestion.metrics_v5_201907', 'congestion.metrics_v5_201908', 'congestion.metrics_v5_201909', 'congestion.metrics_v5_201910', 'congestion.metrics_v5_201911', 'congestion.metrics_v5_201912' ]
for i in month:
    a = a + 1
    print(i)
    with con:
        with con.cursor() as cur:
            sql = '''CREATE TABLE {0} AS
                    with highway as (select distinct segment_id from congestion.segment_links_v5 
                                             inner join (
                                             select routing_grid.* from congestion.routing_grid
                                             join here_gis.streets_att_18_3 on link_id =  (left(id::Text, -1))::numeric
                                             where st_name = 'LAKE SHORE BLVD E' or st_name = 'LAKE SHORE BLVD W' or st_name = 'DON VALLEY PKWY' or st_name = 'HWY-404'or st_name ILIKE '%401%' or st_name = 'GARDINER EXPY'
                                            )a using (link_dir))
                    , tti_prep as  (
                    SELECT a.segment_id,
                    a.datetime_bin::time AS time_bin,
                    COUNT(datetime_bin) AS num_bins,
                    b.seg_length,
                    AVG(a.segment_tt_avg_hc_corr) AS avg_tt_corr,
                    AVG(a.segment_tt_med_hc_corr) AS med_tt_corr,
                    case when highway.segment_id is not null then b.tt_baseline_10pct_corr else b.tt_baseline_25pct_corr end AS baseline_tt_corr,
                    b.tt_baseline_25pct_med_corr AS baseline_tt_med_corr

                    FROM (select * from congestion.tt_segments_30min_v5_2019_af
                    WHERE date_part('month', datetime_bin) = {1} ) a
                    LEFT JOIN congestion.tt_segments_baseline_v5_2019_af b USING (segment_id)
                    left join highway using (segment_id)	

                    WHERE a.segment_tt_avg_all IS NOT NULL
                    GROUP BY segment_id, highway.segment_id, datetime_bin::time, b.seg_length, tt_baseline_10pct_corr, tt_baseline_25pct_corr, tt_baseline_25pct_med, tt_baseline_25pct_med_corr
                    ORDER BY segment_id, datetime_bin::time
                    ) 
                    , bi_prep_am as (
                    SELECT a.segment_id,
                    '08:00:00'::time without time zone AS time_bin,
                    AVG(a.segment_tt_avg_hc_corr) AS avg_tt_corr,
                    PERCENTILE_CONT (0.95) WITHIN GROUP (ORDER BY a.segment_tt_avg_hc_corr ASC) AS pct95_tt_corr, 
                    count(segment_tt_avg_hc_corr) as num_bins

                    FROM (
                    select * from congestion.tt_segments_30min_v5_2019_af
                    WHERE date_part('month', datetime_bin) = {1} and datetime_bin::time <@ '[08:00:00, 08:30:00]'::timerange) a
                    group by segment_id )

                    , bi_prep_pm as (
                    SELECT a.segment_id,
                    '17:00:00'::time without time zone AS time_bin,
                    AVG(a.segment_tt_avg_hc_corr) AS avg_tt_corr,
                    PERCENTILE_CONT (0.95) WITHIN GROUP (ORDER BY a.segment_tt_avg_hc_corr ASC) AS pct95_tt_corr, 
                    count(segment_tt_avg_hc_corr) as num_bins

                    FROM (
                    select * from congestion.tt_segments_30min_v5_2019_af
                    WHERE date_part('month', datetime_bin) = {1} and datetime_bin::time <@ '[17:00:00, 17:30:00]'::timerange) a
                    group by segment_id )	


                    SELECT segment_id, time_bin, tti.num_bins as tti_num_bins,
                    tti.avg_tt_corr, tti.med_tt_corr, tti.baseline_tt_corr, tti.baseline_tt_med_corr, 
                    tti.avg_tt_corr / tti.baseline_tt_corr AS tti_corr,
                    tti.med_tt_corr / tti.baseline_tt_med_corr AS tti_med_corr, 
                    COALESCE((am.pct95_tt_corr- am.avg_tt_corr)/am.avg_tt_corr,(pm.pct95_tt_corr- pm.avg_tt_corr)/pm.avg_tt_corr) as bi,
                    COALESCE(am.num_bins, pm.num_bins) as bi_num_bins

                    from tti_prep tti
                    left join bi_prep_am am using (segment_id, time_bin)
                    left join bi_prep_pm pm using (segment_id, time_bin)
'''.format(i,a)
            cur.execute(sql)

congestion.metrics_v5_201901
congestion.metrics_v5_201902
congestion.metrics_v5_201903
congestion.metrics_v5_201904
congestion.metrics_v5_201905
congestion.metrics_v5_201906
congestion.metrics_v5_201907
congestion.metrics_v5_201908
congestion.metrics_v5_201909
